In [1]:
import numpy as np
import numba
from numba import prange

In [2]:
def arnoldi_iteration(A, b, n, verbose, matvec):
    h = np.zeros((n + 1, n))
    Q = np.zeros((n + 1, A.shape[0]))

    Q[0] = b / np.linalg.norm(b)
    for k in range(1, n + 1):
        v = matvec(A, Q[k - 1])
        for j in range(k):
            h[j, k - 1] = np.dot(Q[j], v)
            v -= h[j, k - 1] * Q[j]
        h[k, k - 1] = np.linalg.norm(v)
        if verbose:
            print(h[k, k - 1])
        Q[k] = v / h[k, k - 1]
    return h

In [3]:
def matvec_simple(A, b):
    n = b.size
    z = np.zeros(n)
    for i in range(n):
        for j in range(n):
            z[i] += A[i][j] * b[j]
    return z

@numba.jit(nopython=True)
def matvec_numba(A, b):
    n = b.size
    z = np.zeros(n)
    for i in range(n):
        for j in range(n):
            z[i] += A[i][j] * b[j]
    return z

@numba.jit(nopython=True, parallel=True)
def matvec_numba_parallel(A, b):
    n = b.size
    z = np.zeros(n)
    for i in prange(n):
        for j in range(n):
            z[i] += A[i][j] * b[j]
    return z

In [4]:
A_test = np.array([[2.0, 2.0], [2.0, 2.0]])
b_test = np.array([2.0, 2.0])
print(matvec_numba(A_test, b_test))
print(matvec_numba_parallel(A_test, b_test))

[8. 8.]
[8. 8.]


In [5]:
n = 5000
k = 4

In [6]:
np.random.seed(seed=1)
A = np.random.rand(n, n)
b = np.random.rand(n)

### Simple matvec

In [7]:
%%time
h = arnoldi_iteration(A, b, k, True, matvec_simple)

1083.1252052635512
23.67790056752565
20.47699083489922
20.650615698615393
CPU times: user 48 s, sys: 7.55 ms, total: 48 s
Wall time: 48 s


### Simple numba matvec

In [8]:
%%time
h = arnoldi_iteration(A, b, k, True, matvec_numba)

1083.1252052635512
23.67790056752565
20.47699083489922
20.650615698615393
CPU times: user 166 ms, sys: 49 µs, total: 166 ms
Wall time: 168 ms


### Paralleled numba matvec

In [9]:
numba.set_num_threads(4)

In [10]:
%%time
h = arnoldi_iteration(A, b, k, True, matvec_numba_parallel)

1083.1252052635512
23.67790056752565
20.47699083489922
20.650615698615393
CPU times: user 245 ms, sys: 3.94 ms, total: 249 ms
Wall time: 65.1 ms


### Multiprocessing

In [11]:
from multiprocessing import Process, Array, Barrier, Queue
import ctypes

In [12]:
def worker(A, x, res, n,  l, r, task_queue, feedback_queue, stop):
    res = np.frombuffer(res, dtype=np.float64)
    x = np.frombuffer(x, dtype=np.float64)
    work = 1
    end = 2
    while True:
        task = task_queue.get() 
        if (task == end):
            break
        res[l : r] = 0.0
        for i in range(l, r):
            for j in range(n):
                res[i] += A[i][j] * x[j]
        feedback_queue.put(1)
        stop.wait() 

In [13]:
def arnoldi_iteration_multiprocessing(A, b, n, verbose, num_proc):
    h = np.zeros((n + 1, n))
    Q = np.zeros((n + 1, A.shape[0]))

    block_size = A.shape[0] // num_proc
    positions = [0 for i in range(num_proc + 1)]
    for i in range(num_proc):
        curr_amount = block_size
        if i < A.shape[0] % num_proc:
            curr_amount += 1
        positions[i + 1] = positions[i] + curr_amount

    work = 1
    end = 2

    task_queue = Queue()
    feedback_queue = Queue()
    stop = Barrier(num_proc)

    # array for res
    v_shared = Array(ctypes.c_double, A.shape[0], lock=False)
    v = np.frombuffer(v_shared, dtype=np.float64)
    
    # array for rhs
    Qj_shared = Array(ctypes.c_double, A.shape[0], lock=False)
    Qj = np.frombuffer(Qj_shared, dtype=np.float64)

    processes = []
    for i in range(num_proc):
        left_ind = positions[i]
        right_ind = positions[i + 1]
        processes.append(Process(target=worker, args=(A, Qj_shared, v_shared, A.shape[0], left_ind, right_ind, task_queue, feedback_queue, stop)))
        processes[i].start()

    def matvec(arr):
        np.copyto(Qj, arr)
        for i in range(num_proc):
            task_queue.put(work)
        for i in range(num_proc):
            feedback_queue.get()

    Q[0] = b / np.linalg.norm(b)

    for k in range(1, n + 1):
        matvec(Q[k - 1])
        for j in range(k):
            h[j, k - 1] = np.dot(Q[j], v)
            v -= h[j, k - 1] * Q[j]
        h[k, k - 1] = np.linalg.norm(v)
        if verbose:
            print(h[k, k - 1])
        Q[k] = v / h[k, k - 1]
    
    def end_processes():
        for i in range(num_proc):
            task_queue.put(end)
        for process in processes:
            process.join()
    
    end_processes()

    return h

In [14]:
%%time
h = arnoldi_iteration_multiprocessing(A, b, k, True, 4)

1083.1252052635512
23.67790056752565
20.47699083489922
20.650615698615393
CPU times: user 6.69 ms, sys: 39.7 ms, total: 46.4 ms
Wall time: 18.6 s


### Results

In [15]:
times_simple = []
for size in np.linspace(100, 2500, 10):
    n = int(size)
    print(n)
    A = np.random.rand(n, n)
    b = np.random.rand(n)
    times = %timeit -o -n 1 -r 1 -q arnoldi_iteration(A, b, 3, False, matvec_simple)
    times_simple.append(times.average)

100
366
633
900
1166
1433
1700
1966
2233
2500


In [16]:
times_numba_simple = []
for size in np.linspace(100, 5000, 10):
    n = int(size)
    print(n)
    A = np.random.rand(n, n)
    b = np.random.rand(n)
    times = %timeit -o -n 5 -r 10 -q arnoldi_iteration(A, b, 3, False, matvec_numba)
    times_numba_simple.append(times.average)

100
644
1188
1733
2277
2822
3366
3911
4455
5000


In [17]:
times_numba_parallel = []
for size in np.linspace(1000, 15000, 10):
    n = int(size)
    print(n)
    A = np.random.rand(n, n)
    b = np.random.rand(n)
    for threads in range(1, 9):
        numba.set_num_threads(threads)
        times = %timeit -o -n 1 -r 5 -q arnoldi_iteration(A, b, 3, False, matvec_numba)
        times_numba_parallel.append(times.average)

1000
2555
4111
5666
7222
8777
10333
11888
13444
15000


In [18]:
times_multiprocessing = []
for size in np.linspace(1000, 10000, 10):
    n = int(size)
    print(n)
    A = np.random.rand(n, n)
    b = np.random.rand(n)
    for threads in range(1, 7):
        print(threads)
        times = %timeit -o -n 1 -r 1 arnoldi_iteration_multiprocessing(A, b, 3, False, threads)
        print(times)
        print(times.average)
        times_multiprocessing.append(times.average)

1000
1
1.48 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.48 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.4756701090000206
2
764 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
764 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
0.7636390920001759
3
624 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
624 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
0.6236950450002041
4
691 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
691 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
0.691175949999888
5
664 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
664 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
0.6638222540000243
6
681 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
681 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
0.680741825000041
2000
1
5.45 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
5.45 s ± 0 ns per loop (mean ± s

In [22]:
import plotly.graph_objects as go

In [27]:
times_numba_parallel = np.array(times_numba_parallel).reshape(10, 8)
matrix_size, threads = np.meshgrid(np.arange(1, 9), np.linspace(1000, 10000, 10))
fig = go.Figure(data=[go.Surface(z=times_numba_parallel, x=matrix_size, y=threads)])
fig.update_layout(title='Numba parallel', autosize=False,
                  width=750, height=750,
                  margin=dict(l=65, r=50, b=65, t=90),
                  scene= go.Scene(
                    xaxis=go.XAxis(title='Amount of threads'),
                    yaxis=go.YAxis(title='Size of matrix'),
                    zaxis=go.ZAxis(title='Time, sec')))
fig.show()

In [26]:
times_multiprocessing = np.array(times_multiprocessing).reshape(10, 6)
matrix_size, threads = np.meshgrid(np.arange(1, 7), np.linspace(1000, 10000, 10))
fig = go.Figure(data=[go.Surface(z=times_multiprocessing, x=matrix_size, y=threads)])
fig.update_layout(title='Multiprocessing', autosize=False,
                  width=750, height=750,
                  margin=dict(l=65, r=50, b=65, t=90),
                  scene= go.Scene(
                    xaxis=go.XAxis(title='Amount of threads'),
                    yaxis=go.YAxis(title='Size of matrix'),
                    zaxis=go.ZAxis(title='Time, sec')))
fig.show()

In [25]:
times_multiprocessing

array([[  1.47567011,   0.76363909,   0.62369505,   0.69117595,
          0.66382225,   0.68074183],
       [  5.45228072,   2.90468711,   2.46140772,   2.17945366,
          2.4215654 ,   2.47744397],
       [ 12.49399783,   6.44467422,   5.45860021,   4.75228071,
          5.41076632,   5.16641679],
       [ 21.8046067 ,  11.66299344,   9.64111098,   8.71575945,
          9.37159543,   9.17949551],
       [ 38.43692184,  18.61309146,  15.25001444,  13.36552497,
         14.51170129,  13.84325776],
       [ 49.3987227 ,  26.84751287,  21.69317729,  19.06205437,
         20.98288583,  19.68929754],
       [ 67.13829803,  37.30431185,  29.93657643,  25.66377069,
         28.45941432,  26.86587356],
       [ 88.05311543,  48.66111839,  39.58149769,  34.40538197,
         37.50511762,  35.85853749],
       [112.10682752,  64.66438079,  51.50051169,  39.20042292,
         44.64466423,  41.46047052],
       [135.08775769,  74.69593599,  57.02929703,  48.67774325,
         57.16405978,  55.8